In [ ]:
import duckdb

# connect to duckdb database
con = duckdb.connect("../data/analysis.duckdb")

In [ ]:

con.sql(
"""
COPY (
  SELECT
    Area,
    Code,
    Year,
    Total_Wages,
    Real_Total_Wages,
    Total_Permits,
    Change_Real_Wage,
    Change_Permit,
    Wage_Index,
    Permit_Index,
    Zoning_Pressure
  FROM annual_metrics
  ORDER BY Code, Year
)
TO '../data/derived/annual_metrics.parquet'
(FORMAT PARQUET);
"""
)

con.sql(
"""
COPY (
  SELECT
    Area,
    Code,
    Year,
    Real_Total_Wages,
    Total_Permits,
    Cumul_Wage_Index,
    Cumul_Permit_Index,
    Structural_Gap
  FROM cumulative_metrics
  ORDER BY Code, Year
)
TO '../data/derived/cumulative_metrics.parquet'
(FORMAT PARQUET);
""")


In [ ]:
df = con.sql("""SELECT area, zoning_pressure  
FROM annual_metrics  
WHERE Year = '2023'  
ORDER BY Zoning_Pressure DESC; """).df()
print(df)

In [ ]:
df = con.sql("""
SELECT year, area, structural_gap,  
RANK() OVER (PARTITION BY year ORDER BY structural_gap DESC) AS gap_rank  
FROM cumulative_metrics  
WHERE year > 2014
ORDER BY year, gap_rank """ ).df()
print(df.head(10))

In [ ]:
df = con.sql("""                
SELECT area, AVG(zoning_pressure) AS avg_pressure  
FROM annual_metrics  
GROUP BY area  
ORDER BY avg_pressure DESC;
""").df()
print(df)

In [ ]:
print(con.sql('select * from dim_metro_full where Code IN (42660, 38900, 33460, 12420, 47900,15680)').df())
print(con.sql('SELECT COUNT(*) AS num_metros FROM dim_metro_full').df())

In [ ]:
print(con.sql('''
SELECT LENGTH(CAST(ABS(Code) AS TEXT)) AS NumberOfDigits 
FROM dim_metro_full 
WHERE NumberOfDigits <> 5;
''').df())


In [ ]:
df = con.sql("""
    SELECT Code, Year, Quarter, COUNT(*) AS months_in_qtr  
    FROM permits_metrics  
    GROUP BY Code, Year, Quarter  
    HAVING COUNT(*) NOT IN (3, 0); 
""").df()
print(df)

In [ ]:
print(con.sql("""
SELECT Code, Area, Title 
FROM dim_metro_full 
WHERE Title 
LIKE '%FL%' OR Title LIKE '%CO%' 
ORDER BY Title Asc, Area Asc;
""").df())


In [ ]:
df = con.execute('''
CREATE OR REPLACE TABLE permits_metrics_qtr AS  
SELECT  
  Code, 
  Year, 
  Quarter, 
  SUM("Total_Permits") AS Total_Permits_Qtr  
FROM permits_metrics  
GROUP BY Code, Year, Quarter; 
''').df()

In [ ]:
df = con.sql("SELECT * from permits_metrics_qtr LIMIT 5;").df()
print(df)

In [ ]:
df = con.execute('''
    CREATE OR REPLACE TABLE quarterly_metrics AS  
    SELECT  
      w.Area AS Area,  
      w.Code AS Code,  
      w.Year AS Year,  
      w.Quarter AS Quarter,  
      w."Total_Wages" AS Total_Wages,  
      p.total_permits_qtr as Total_Permits,  
    FROM wages_metrics AS w  
    JOIN permits_metrics_qtr AS p  
    Using (Code, Year, Quarter)  '''
).df()

In [ ]:
df = con.sql("SELECT * FROM quarterly_metrics;").df()
print(df)

In [ ]:
# base year exists per metro
print(con.sql(
"""SELECT Code, COUNT(*) AS n
FROM cumulative_metrics
WHERE Year = 2015
GROUP BY Code
HAVING n <> 1;
""").df())

#no missing base data
print(con.sql(
"""
SELECT *
FROM cumulative_metrics
WHERE Base_Wage IS NULL OR Base_Permits IS NULL OR Base_Permits = 0;
""").df())

In [ ]:
print(con.sql('select table_name, column_name, data_type from information_schema."columns";').df())


In [ ]:
df = con.sql(''' 
SELECT  
  w.Area AS area,  
  w.Code AS code,  
  w.Year AS year,  
  w.Quarter AS quarter,  
  w.Total_Wages AS total_wages,  
  p.Month AS month,  
  p.Total_Permits AS total_permits  
FROM wages_metrics AS w  
INNER JOIN permits_metrics as p  
USING (Code, Year, Quarter)  
-- WHERE w.Code = 47900  
ORDER BY  
  area DESC,  
  year DESC,  
  month DESC  
-- LIMIT 12; ''').df()

print(f"top 10:\n {df.head(10)}\n\nnumber of rows: {len(df)}")

In [ ]:
con.close()